# AdaptiveMD

## Example 5 - Working with projects

### 0. Imports

In [1]:
import sys, os

In [2]:
from adaptivemd import Project, Event, FunctionalEvent, Trajectory, RestartFile

Let's open our `test` project by its name. If you completed the previous example this should all work out of the box.

In [3]:
project = Project('example-worker')

Open all connections to the `MongoDB` and `Session` so we can get started.

Let's see where we are. These numbers will depend on whether you run this notebook for the first time or just continue again. Unless you delete your project it will accumulate models and files over time, as is our ultimate goal.

In [4]:
print project.tasks

print project.trajectories
print project.models

<StoredBundle with 78 file(s) @ 0x10f08e4d0>
<ViewBundle with 49 file(s) @ 0x10f08e590>
<StoredBundle with 16 file(s) @ 0x10f08e410>


Now restore our old ways to generate tasks by loading the previously used generators.

In [5]:
engine = project.generators['openmm']
modeller = project.generators['pyemma']
pdb_file = project.files['initial_pdb']

### 1. Task Analysis

In [6]:
print project.tasks.one.description

Task: PyEMMAAnalysis [created]

Required : []
Output : []
Modified : []

<pretask>
Transfer('file://{}/_rpc_input_d0dfa6e7-4e4e-48ba-903c-f1f4c7cf7cd3.json' > 'worker://input.json)
Link('staging:///_run_.py' > 'worker://_run_.py)
Link('staging:///alanine.pdb' > 'worker://input.pdb)
python _run_.py
Transfer('worker://output.json' > 'file://{}/_rpc_output_d0dfa6e7-4e4e-48ba-903c-f1f4c7cf7cd3.json)
<posttask>


In [7]:
task = modeller.task_run_msm_files(list(project.trajectories)[0:10])
project.queue(task)

Added file PythonTask


In [44]:
w = project.workers.last
print w.hostname, w.state, w.command, w.current

Stevie.fritz.box down None None


In [42]:
for l in  project.logs:
    if l.title == 'execution error':
        print l.message

failed to create file before staging worker://output.dcd
failed to create file before staging worker://output.json
failed to create file before staging worker://output.json
failed to create file before staging worker://output.json


In [20]:
from adaptivemd.generator import rpc_exec

In [22]:
rpc_exec.path

'/Users/jan-hendrikprinz/Studium/git/adaptive-sampling/package/adaptivemd/scripts/_run_.py'

In [19]:
for f in project.files:
    if f.basename == '_run_.py':
        print f.short, f.get_file()

file://{}/_run_.py """
Python script for the RPC Python call for Adaptive MD

This executes a function remotely and expects an `input.json` file to contain
a reference to the function and module as well as arguments and keyword arguments
"""


import importlib
from adaptivemd.mongodb import ObjectJSON

simplifier = ObjectJSON()

with open('input.json', 'r') as f:
    data = simplifier.from_json(f.read())

parts = data['function'].split('.')

fnc = importlib.import_module('.'.join(parts[:-1]))

fnc = getattr(fnc, parts[-1])

result = fnc(*data['args'], **data['kwargs'])

with open('output.json', 'w') as f:
    f.write(simplifier.to_json(result))



In [23]:
import subprocess

In [55]:
%%file test.sh
export
echo $FILE

Overwriting test.sh


In [62]:
q = subprocess.Popen(['/bin/bash', 'test.sh'], stdout=subprocess.PIPE)

In [63]:
stdout_value = q.communicate()[0]

In [64]:
print stdout_value

declare -x Apple_PubSub_Socket_Render="/private/tmp/com.apple.launchd.1sh4m1KkVa/Render"
declare -x CLICOLOR="1"
declare -x DISPLAY="/private/tmp/com.apple.launchd.TYmY93GPXR/org.macosforge.xquartz:0"
declare -x GIT_PAGER="cat"
declare -x GREP_OPTIONS="--color=auto"
declare -x HOME="/Users/jan-hendrikprinz"
declare -x JPY_PARENT_PID="5251"
declare -x LANG="de_DE.UTF-8"
declare -x LOGNAME="jan-hendrikprinz"
declare -x LSCOLORS="GxFxCxDxBxegedabagaced"
declare -x MPLBACKEND="module://ipykernel.pylab.backend_inline"
declare -x OLDPWD
declare -x OPENMM_DIR="/Users/jan-hendrikprinz/anaconda/pkgs/openmm-7.0.1-py27_0"
declare -x OPENMM_INCLUDE_PATH="/Users/jan-hendrikprinz/anaconda/pkgs/openmm-7.0.1-py27_0/include"
declare -x OPENMM_LIB_PATH="/Users/jan-hendrikprinz/anaconda/pkgs/openmm-7.0.1-py27_0/lib"
declare -x OPENMM_PLUGIN_DIR="/Users/jan-hendrikprinz/anaconda/pkgs/openmm-7.0.1-py27_0/lib/plugins"
declare -x PAGER="cat"
declare -x PATH="/Users/jan-hendrikprinz/anaconda/bin:/Users/jan-he

### 1. Run simulations

In [10]:
task = project.tasks.one

In [17]:
[t.source for t in task.output_staging]

[False]

Before we talk about adaptivity, let's have a look at possibilities to generate trajectories.

We assume that you successfully ran a first trajectory using a worker. Next, we talk about lot's of ways to generate new trajectories.

#### Trajectories from a pdb

You will do this in the beginning. Remember we already have a PDB stored from setting up the engine. if you want to start from this configuration do as before

1. get the trajectory you want
2. make a task
3. submit the task

##### The `Trajectory` object

In [7]:
file_name = next(project.traj_name)

trajectory = Trajectory(
    location=file_name,                          # this creates a new filename
    frame=pdb_file,                              # initial frame is the PDB
    length=100,                                  # length is 100 frames
    restart=RestartFile(file_name + '.restart')  # save also the .restart file
)

Since this is tedious to write there is a shortcut

In [8]:
trajectory = project.new_trajectory(
    frame=pdb_file,
    length=100,
    restart=True,     # if True it will create the RestartFile(`{}.restart`)
    number=1          # if more then one you get a list of trajectories
)

##### The `Task` object

In [9]:
task_run = engine.task_run_trajectory(trajectory)

This was easy, but we can do some interesting stuff. Since we know the trajectory will exist now we can also extend by some frames.

In [10]:
task_extend = engine.task_extend_trajectory(trajectory, 50)

The only problem is to make sure the tasks are run in the correct order. This would not be a problem if the worker will run tasks in the order they are place in the queue, but that defeats the purpose of parallel runs. Therefore an extended tasks knows that is depends on the existance of the source trajectory. The worker will hence only run a trajectory, once the source exists.

In [11]:
project.queue(task_run, task_extend)

Added file TrajectoryGenerationTask
Added file TrajectoryExtensionTask


#### Check the results

For a seconds let's see if everything went fine.

In [12]:
for t in project.trajectories:
    print t.path, t.length

/projects/example-worker/trajs/00000000.dcd 100
/projects/example-worker/trajs/00000001.dcd 150


If this works, then you should see one 100 frame trajectory from the setup (first example) and a second 150 length trajectory that we just generated by running 100 frames and extending it by another 50.

If not, there might be a problem or (more likely) the tasks are not finished yet. Just try the above cell again and see if it changes to the expected output.

Let's do something stupid and produce an error by using a wrong initial pdb file.

In [13]:
trajectory = project.new_trajectory(engine['system_file'], 100)
task = engine.task_run_trajectory(trajectory)
project.queue(task)

Added file TrajectoryGenerationTask


In [14]:
for t in project.trajectories:
    print t.path, t.length

/projects/example-worker/trajs/00000000.dcd 100
/projects/example-worker/trajs/00000001.dcd 150


Well, nothing changed obviously and we expect it to fail. So let's inspect what happened.

In [15]:
task.state

u'created'

It failed, well, we kind of knew that. No suprise here, but why? Let's look at the stdout and stderr

In [16]:
print task.stdout

None


In [17]:
print task.stderr

None


We see, what we expect. In `openmmrun.py` the openmm executable it could not load the pdb file. 

> *NOTE* If your worker dies for some reason, it will not set a STDOUT or STDERR. If you think that your task should be able to execute, then you can do `task.state = 'created'` and reset it to be accessible to workers. This is NOT recommended, just to explain how this works. Of course you need a new worker anyway.

#### What else

If you have the trajectory object and want to just create the trajectory, you can also put this in the queue. A `Task` object is generated for you, BUT you will have no direct access to the task as in this example. Do whatever you need

In [19]:
project.queue(project.new_trajectory(pdb_file, 100))

Added file TrajectoryGenerationTask


### Trajectories from other trajectories

This will be the most common case. At least in any remote kind of adaptivity you will not start always from the same position or extend. You want to pick any exisiting frame and continue from there. So, let's do that.

First we get a trajectory. Every bundle in the project (e.g. `.trajectories`, `.models`, `.files`, `.tasks`) acts like an enhanced set. You can iterate over all entries as we did before, and you can get one element, which usually is the first stored, but not always. For now that is enough

In [21]:
trajectory = project.trajectories.one
print trajectory.path, len(trajectory)

/projects/example-worker/trajs/00000000.dcd 100


Good, at least 100 frames. We pick, say, frame at index 28 (which is the 29th frame, we start counting at zero) using the way you pick an element from a python list (which is almost what a `Trajectory` represents, a list of frames)

In [24]:
frame = trajectory[28]
print frame

Frame(00000000.dcd[28])


To run a trajectory just use the frame as the initial frame.

In [28]:
trajectory = project.new_trajectory(frame, 100)
task = engine.task_run_trajectory(trajectory)
project.queue(task)

Added file TrajectoryGenerationTask


Btw, you can wait until something happens using `project.wait_until(condition)`. This is not so useful in notebooks, but in scripts it does. `condition` here is a function that evaluates to `True` or `False`. it will be tested in regular intervals and once it is `True` the function returns.

In [29]:
project.wait_until(task.is_done)

Each `Task` has a function `is_done` that you can use. It will return once a task is done. That means it either failed or succeeded or was cancelled. Basically when it is not queued anymore.

If you want to run adaptively, _all you need to do_ is to figure out where to start new simulations from and use the methods provided to run these.

### `Model` tasks

A model generating task work similar to trajectories. You create the generator with options (so far, this will become more complex in the future) and then you create a `Task` from passing it a list of trajectories to be analyzed.

In [52]:
task = modeller.task_run_msm_files(list(project.trajectories))
project.queue(task)

Added file PythonTask


In [53]:
for m in project.models:
    print m

So we generated one model. The `Model` objects contain (in the base version) only a `.data` attribute which is a dictionary of information about the generated model.

In [56]:
model = project.models.one
model.data.keys()

['clustering', 'input', 'msm', 'tica']

In [57]:
print model.data['msm']['P']

[[  8.83248730e-01   3.77172257e-02   0.00000000e+00   6.95497425e-02
    9.48430165e-03]
 [  3.68854736e-02   9.20529799e-01   0.00000000e+00   4.25847272e-02
    0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   8.94736839e-01   1.05263161e-01
    0.00000000e+00]
 [  4.07748643e-02   2.55290847e-02   7.82122887e-02   8.54748601e-01
    7.35161132e-04]
 [  8.61682481e-02   0.00000000e+00   0.00000000e+00   1.13927206e-02
    9.02439031e-01]]


### Pick frames automatically

The last thing that is implemented is a function that can utilize models to decide which frames are better to start from. The simplest one will use the counts per state, take the inverse and use this as a distribution.

In [64]:
project.find_ml_next_frame(4)

[Frame(00000008.dcd[99]),
 Frame(00000001.dcd[88]),
 Frame(00000001.dcd[99]),
 Frame(00000000.dcd[85])]

So you can pick states according to the newest (last) model. (This will be moved to the Brain). And since we want trajectories with these frames as starting points there is also a function for that

In [65]:
trajectories = project.new_ml_trajectory(length=100, number=4)
trajectories

[Trajectory(Frame(00000001.dcd[102]) >> 00000013.dcd[0..100]),
 Trajectory(Frame(00000000.dcd[69]) >> 00000014.dcd[0..100]),
 Trajectory(Frame(00000000.dcd[17]) >> 00000015.dcd[0..100]),
 Trajectory(Frame(00000001.dcd[95]) >> 00000016.dcd[0..100])]

Let's submit these before we finish this notebook with a quick discussion of workers

In [66]:
project.queue(trajectories)

Added file TrajectoryGenerationTask
Added file TrajectoryGenerationTask
Added file TrajectoryGenerationTask
Added file TrajectoryGenerationTask


That's it.

### What about workers

Worker are the instances that execute tasks for you. If you did not stop the worker it will still be running and you can check its state

In [35]:
from adaptivemd import DT

DT is a little helper to convert time stamps into something readable.

In [68]:
for w in project.workers:
    print '[%s:%s] %s:%s runs `%s` tasks' % (w.state, DT(w.seen).time, w.hostname, w.cwd, w.generators)

[down:23:23:50] Stevie.fritz.box:/Users/jan-hendrikprinz runs `['openmm']` tasks
[down:23:14:55] Stevie.fritz.box:/Users/jan-hendrikprinz runs `['openmm']` tasks
[running:23:44:11] Stevie.fritz.box:/Users/jan-hendrikprinz runs `None` tasks


Okay, the worker is running, was last reporting its heartbeat at ... and has a hostname and current working directory (where it was executed from). The generators specify which tasks from some generators are executed. If it is `None` then the worker runs all tasks it finds. You can use this to run specific workers for models and some for trajectory generation.

You can also controle it remotely by sending it a command. `shutdown` will shut it down for you.

In [50]:
# project.workers.last.command = 'shutdown'

Afterwards you need to restart you worker to continue.

In [69]:
project.close()